## Langkah 1 - Load Data

In [ ]:
# Load data
import pandas as pd
data = pd.read_csv('iris.csv')
data.head()

## Langkah 2 - Eksplorasi Data
Cek struktur data informasi deskriptif data

In [ ]:
data.info()
data.describe()
data['species'].value_counts()

## Langkah 3 - Visualisasi Data
Lakukan visualisasi data untuk mengetahui distribusi dan korelasi setiap fitur terhadap label.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.pairplot(data, hue='species')
plt.show()

## Langkah 4 - Preprocessing
Pada tahap ini, kita akan memisahkan antara label dengan fitur yang akan digunakan. Selain itu, untuk alasan pembelajaran, kita akan melakukan standardisasi dari fitur yang akan digunakan. 

> Pada kasus Iris Dataset, seluruh fitur sudah dalam satuan yang sama yaitu cm.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = data.iloc[:, :-1]   # semua kolom kecuali label
y = data.iloc[:, -1]    # kolom label terakhir

# Split data (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardisasi
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Langkah 5 - Buat Model kNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Tentukan nilai K (misalnya 3)
knn = KNeighborsClassifier(n_neighbors=3)

# Latih model
knn.fit(X_train, y_train)

## Langkah 6 - Evaluasi Model kNN
Pada tahap ini kita akan melakukan evaluasi terhadap model kNN yang telah dibuat sebelumnya. Metrik utama yang akan digunakan adalah akurasi dan detail analisis menggunakan confusion metrics.

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

y_pred = knn.predict(X_test)
print("Akurasi:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Laporan Klasifikasi:\n", classification_report(y_test, y_pred))

Di dapatkan hasil:

```
Akurasi: 1.0
Confusion Matrix:
 [[19  0  0]
 [ 0 13  0]
 [ 0  0 13]]
Laporan Klasifikasi:
               precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        19
  versicolor       1.00      1.00      1.00        13
   virginica       1.00      1.00      1.00        13

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45
```

Dapat dilihat bahwa model dapat bekerja dengan performa sempurna pada data testing. Namun apakah memang nilai k=3 adalah nilai yang terbaik? Selanjutnya kita akan mengevaluasi setiap nilai k.

In [ ]:
acc = []
for k in range(1, 11):
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(X_train, y_train)
    acc.append(model.score(X_test, y_test))

plt.plot(range(1, 11), acc, marker='o')
plt.title('Nilai K vs Akurasi')
plt.xlabel('Nilai K')
plt.ylabel('Akurasi')
plt.show()

Dapat dilihat bahwa performa data test mendapatkan nilai sempurna pada k=3 dilanjutkan dengan k=6 hingga k=10. Meskipun demikian, perlu dicatat bahwa kNN tidak pernah menyimpan bobot hasil training, proses klasifikasi dilakukan secara langsung sesuai dengan jumlah tetangga sehingga hasil ini masih perlu dibandingkan dengan hasil data training.